In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from argparse import Namespace
import misc.logging_utils as logging_utils

args = Namespace()
logger = logging_utils.get_ipython_logger()

In [48]:
import numpy as np
import networkx as nx

import misc.automl_utils as automl_utils
import misc.parallel as parallel
import misc.utils as utils

import as_auto_sklearn.as_asl_command_line_utils as clu
import as_auto_sklearn.as_asl_filenames as filenames
import as_auto_sklearn.as_asl_utils as as_asl_utils
from as_auto_sklearn.as_asl_ensemble import ASaslPipeline

from as_auto_sklearn.presolver_scheduler import PresolverScheduler
from as_auto_sklearn.sequential_feature_step_selector import SequentialFeatureStepSelector

In [94]:
logger.setLevel("INFO")

In [4]:
args.training_scenarios_dir = "/mldb/oasc_scenarios/train/"
training_scenarios = automl_utils.load_all_scenarios(args.training_scenarios_dir)

scenario = training_scenarios['Svea']
feature_dependency_graph = automl_utils.extract_feature_step_dependency_graph(scenario)
feature_topo_order = nx.topological_sort(feature_dependency_graph)

INFO     : Read ASlib scenario: /mldb/oasc_scenarios/train/Bado
INFO     : Read /mldb/oasc_scenarios/train/Bado/description.txt
INFO     : Read /mldb/oasc_scenarios/train/Bado/algorithm_runs.arff
INFO     : Read /mldb/oasc_scenarios/train/Bado/feature_values.arff
INFO     : Read /mldb/oasc_scenarios/train/Bado/feature_runstatus.arff
INFO     : Read /mldb/oasc_scenarios/train/Bado/feature_costs.arff
INFO     : Read /mldb/oasc_scenarios/train/Bado/cv.arff
DEBUG    : Replace all runtime data with PAR10 values for non-OK runs
INFO     : Read ASlib scenario: /mldb/oasc_scenarios/train/Camilla
INFO     : Read /mldb/oasc_scenarios/train/Camilla/description.txt
DEBUG    : Since we optimize quality, we use runtime cutoff of 1.
INFO     : Read /mldb/oasc_scenarios/train/Camilla/algorithm_runs.arff
INFO     : Read /mldb/oasc_scenarios/train/Camilla/feature_values.arff
       'instance_85'],
      dtype='object', name='instance_id')
INFO     : Read /mldb/oasc_scenarios/train/Camilla/feature_runsta

INFO     : Read /mldb/oasc_scenarios/train/Quill/feature_costs.arff
INFO     : Read /mldb/oasc_scenarios/train/Quill/cv.arff
DEBUG    : Replace all runtime data with PAR10 values for non-OK runs
INFO     : Read ASlib scenario: /mldb/oasc_scenarios/train/Sora
INFO     : Read /mldb/oasc_scenarios/train/Sora/description.txt
INFO     : Read /mldb/oasc_scenarios/train/Sora/algorithm_runs.arff
INFO     : Read /mldb/oasc_scenarios/train/Sora/feature_values.arff
INFO     : Read /mldb/oasc_scenarios/train/Sora/feature_runstatus.arff
INFO     : Read /mldb/oasc_scenarios/train/Sora/feature_costs.arff
INFO     : Read /mldb/oasc_scenarios/train/Sora/cv.arff
DEBUG    : Replace all runtime data with PAR10 values for non-OK runs
INFO     : Read ASlib scenario: /mldb/oasc_scenarios/train/Svea
INFO     : Read /mldb/oasc_scenarios/train/Svea/description.txt
INFO     : Read /mldb/oasc_scenarios/train/Svea/algorithm_runs.arff
INFO     : Read /mldb/oasc_scenarios/train/Svea/feature_values.arff
       'insta

In [44]:
args.num_cpus = 1
args.folds = range(1,11)

automl_utils.add_automl_values_to_args(args, total_training_time=20)

scenario = automl_utils.load_scenario("/mldb/oasc_scenarios/train/Sora/", return_name=False)


INFO     : Read ASlib scenario: /mldb/oasc_scenarios/train/Sora/
INFO     : Read /mldb/oasc_scenarios/train/Sora/description.txt
INFO     : Read /mldb/oasc_scenarios/train/Sora/algorithm_runs.arff
INFO     : Read /mldb/oasc_scenarios/train/Sora/feature_values.arff
INFO     : Read /mldb/oasc_scenarios/train/Sora/feature_runstatus.arff
INFO     : Read /mldb/oasc_scenarios/train/Sora/feature_costs.arff
INFO     : Read /mldb/oasc_scenarios/train/Sora/cv.arff


In [41]:
feature_dependency_graph.edges()

[('step_1', 'step_10'),
 ('step_1', 'step_12'),
 ('step_1', 'step_15'),
 ('step_1', 'step_3'),
 ('step_1', 'step_4'),
 ('step_1', 'step_6'),
 ('step_1', 'step_7'),
 ('step_1', 'step_8')]

In [54]:

logger.setLevel('INFO')
sfss = SequentialFeatureStepSelector(args)
sfss_fit = sfss.fit(scenario)
sfss.cur_feature_steps_, sfss.cur_par10_

INFO     : SFSS]: *** evaluating feature steps: ['step_1'] ***
INFO     : [SFSS]: evaluating fold: 8
INFO     : [SFSS]: evaluating fold: 9
INFO     : [SFSS]: evaluating fold: 10
INFO     : 
 feature_steps: ['step_1']
 min_par10: 9730.208984962406
 total_timeouts: 75
 total_solved: 324
 total_solver_times: 507377.677
 total_feature_times: 302.88800000000003
INFO     : SFSS]: *** evaluating feature steps: ['step_11'] ***
INFO     : [SFSS]: evaluating fold: 8
INFO     : [SFSS]: evaluating fold: 9
INFO     : [SFSS]: evaluating fold: 10
INFO     : 
 feature_steps: ['step_11']
 min_par10: 10241.267130325814
 total_timeouts: 79
 total_solved: 320
 total_solver_times: 531706.2289999999
 total_feature_times: 5150.5599999999995
INFO     : SFSS]: *** evaluating feature steps: ['step_13'] ***
INFO     : [SFSS]: evaluating fold: 8
INFO     : [SFSS]: evaluating fold: 9
INFO     : [SFSS]: evaluating fold: 10
INFO     : 
 feature_steps: ['step_13']
 min_par10: 11233.769117794487
 total_timeouts: 87
 t

(['step_9'], 9684.2301453634082)

In [57]:
args.config = "/prj/oasc2017/oasc.yaml"

args.training_scenario = "/mldb/oasc_scenarios/train/Sora/"
args.testing_scenario = "/mldb/oasc_scenarios/test/Sora/"

required_keys = ['base_path']
config = as_asl_utils.load_config(args.config, required_keys)


INFO     : Reading config file


In [58]:
selector_type = "rf-ensemble"
selector_filename = filenames.get_feature_selector_filename(
    config['base_path'],
    selector_type,
    note=config.get('note')
)

sfss_fit.dump(selector_filename)

AttributeError: 'SequentialFeatureStepSelector' object has no attribute 'dump'

In [60]:
selected_features = automl_utils.extract_feature_names(
    scenario, 
    sfss.cur_feature_steps_
)


In [65]:
args.total_training_time = 300

pipeline = ASaslPipeline(
    args,
    features=selected_features
)

pipeline_fit = pipeline.fit(scenario)

Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (299.522756)
[WARNING] [2017-08-29 13:17:48,534:AutoMLSMBO(8675309)::6ceb4c5678a58cc49c975f365889ee16] Could not find meta-data directory /home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/autosklearn/metalearning/files/mean_squared_error_regression_dense


You are already timing task: index_run2
You are already timing task: index_run2
You are already timing task: index_run2
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/autosklearn/smbo.py:737: RuntimeWarning: divide by zero encountered in true_divide
  (1. - dataset_minimum))
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/autosklearn/smbo.py:737: RuntimeWarning: invalid value encountered in true_divide
  (1. - dataset_minimum))
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/autosklearn/smbo.py:738: RuntimeWarning: invalid value encountered in greater
  Y_cfg[:, 0][Y_cfg[:, 0] > 2] = 2
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a

/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/autosklearn/smbo.py:737: RuntimeWarning: divide by zero encountered in true_divide
  (1. - dataset_minimum))
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/autosklearn/smbo.py:737: RuntimeWarning: invalid value encountered in true_divide
  (1. - dataset_minimum))
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/autosklearn/smbo.py:738: RuntimeWarning: invalid value encountered in greater
  Y_cfg[:, 0][Y_cfg[:, 0] > 2] = 2
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:

Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (299.617040)
[WARNING] [2017-08-29 13:22:44,618:AutoMLSMBO(8675309)::6ceb4c5678a58cc49c975f365889ee16] Could not find meta-data directory /home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/autosklearn/metalearning/files/mean_squared_error_regression_dense


You are already timing task: index_run2
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/autosklearn/smbo.py:737: RuntimeWarning: divide by zero encountered in true_divide
  (1. - dataset_minimum))
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/autosklearn/smbo.py:737: RuntimeWarning: invalid value encountered in true_divide
  (1. - dataset_minimum))
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/autosklearn/smbo.py:738: RuntimeWarning: invalid value encountered in greater
  Y_cfg[:, 0][Y_cfg[:, 0] > 2] = 2
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packag

...

You are already timing task: index_run7
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/autosklearn/smbo.py:737: RuntimeWarning: divide by zero encountered in true_divide
  (1. - dataset_minimum))
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/autosklearn/smbo.py:737: RuntimeWarning: invalid value encountered in true_divide
  (1. - dataset_minimum))
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/autosklearn/smbo.py:738: RuntimeWarning: invalid value encountered in greater
  Y_cfg[:, 0][Y_cfg[:, 0] > 2] = 2
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/nle5231/.virtualenvs/automl/lib/python3.6/site-packag

Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (299.475580)
[WARNING] [2017-08-29 14:07:15,434:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2017-08-29 14:07:15,434:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger


You are already timing task: index_run2
You are already timing task: index_run2
You are already timing task: index_run2
You are already timing task: index_run2


[WARNING] [2017-08-29 14:08:54,128:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2017-08-29 14:08:54,128:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger


You are already timing task: index_run3
You are already timing task: index_run3
You are already timing task: index_run3
You are already timing task: index_run3
You are already timing task: index_run3


In [66]:

model_type = "as-asl-pipeline"
model_filename = filenames.get_model_filename(
    config['base_path'],
    model_type,
    note=config.get('note')    
)

NotFittedError: This ASaslPipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [ ]:
pipeline_fit.

In [75]:
p.pipeline_

Pipeline(steps=[('feature_selector', ColumnSelector(cols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])), ('nss', <misc.nan_standard_scaler.NaNStandardScaler object at 0x7f2de2a067b8>), ('selector', <as_auto_sklearn.as_asl_ensemble.ASaslEnsemble object at 0x7f2de2a065c0>)])

In [90]:
from as_auto_sklearn.as_asl_ensemble import ASaslPipeline
p = ASaslPipeline(args, features=pipeline_fit.features, feature_steps=sfss.cur_feature_steps_)
p.pipeline_ = pipeline_fit.pipeline_
p.dump(model_filename)

PicklingError: Can't pickle <class 'as_auto_sklearn.as_asl_ensemble.ASaslEnsemble'>: it's not the same object as as_auto_sklearn.as_asl_ensemble.ASaslEnsemble

In [ ]:

schedules = p.create_solver_schedules(scenario)

In [92]:
logger.setLevel("DEBUG")

validator = Validator()
stat = validator.validate(
    schedules=schedules,
    test_scenario=scenario,
    show=True
)


AttributeError: 'Validator' object has no attribute 'validate'

PicklingError: Can't pickle <class 'as_auto_sklearn.as_asl_ensemble.ASaslPipeline'>: it's not the same object as as_auto_sklearn.as_asl_ensemble.ASaslPipeline

In [31]:
4196/(226+8)

17.931623931623932